In [1]:
!pip install flair

     |████████████████████████████████| 322 kB 14.7 MB/s 
     |████████████████████████████████| 19.7 MB 92.4 MB/s 
     |████████████████████████████████| 788 kB 56.9 MB/s 
     |████████████████████████████████| 1.2 MB 48.9 MB/s 
     |████████████████████████████████| 3.5 MB 38.1 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 981 kB 47.2 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 48 kB 7.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 596 kB 64.9 MB/s 
     |████████████████████████████████| 895 kB 66.2 MB/s 
     |████████████████████████████████| 6.8 MB 53.6 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9692 sha256=72ac1017d272ca09b84b7b55a2fd6bcc4d39c0d06097

## Carregando dados

In [22]:
import re

from torch.optim.adamax import Adamax

from flair.datasets import ColumnCorpus, DataLoader
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, CharacterEmbeddings, BytePairEmbeddings
from flair.data import Corpus
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [3]:
!gdown --id 1mZfnBPCPSV0ZjVClLYZoxHzacZZxU5ZG

Downloading...
From: https://drive.google.com/uc?id=1mZfnBPCPSV0ZjVClLYZoxHzacZZxU5ZG
To: /content/en.zip
100% 2.90M/2.90M [00:00<00:00, 54.0MB/s]


In [4]:
!unzip en.zip

Archive:  en.zip
  inflating: secs0-18-training       
  inflating: secs19-21-development   
  inflating: secs22-24-testing       


In [5]:
sep = '_'
with open('secs0-18-training', 'r') as file:
  with open('train.corpus', 'w') as rfile:
    for line in file:
        for pair in re.split(' ', line):
            if len(pair) > 2:
                token, tag = pair.split(sep)
                tag = tag.strip()
                rfile.write(token + ' ' + tag+'\n')
        rfile.write('\n')

In [6]:
sep = '_'
with open('secs19-21-development', 'r') as file:
  with open('dev.corpus', 'w') as rfile:
    for line in file:
        for pair in re.split(' ', line):
            if len(pair) > 2:
                token, tag = pair.split(sep)
                tag = tag.strip()
                rfile.write(token + ' ' + tag+'\n')
        rfile.write('\n')

In [7]:
sep = '_'
with open('secs22-24-testing', 'r') as file:
  with open('test.corpus', 'w') as rfile:
    for line in file:
        for pair in re.split(' ', line):
            if len(pair) > 2:
                token, tag = pair.split(sep)
                tag = tag.strip()
                rfile.write(token + ' ' + tag+'\n')
        rfile.write('\n')

In [8]:
columns = {0: 'text', 1: 'pos'}

data_folder = './'

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.corpus',
                              dev_file='dev.corpus',
                              test_file='test.corpus')

2022-02-15 13:50:35,964 Reading data from .
2022-02-15 13:50:35,971 Train: train.corpus
2022-02-15 13:50:35,974 Dev: dev.corpus
2022-02-15 13:50:35,981 Test: test.corpus


In [9]:
label_type = 'pos'
label_dict = corpus.make_label_dictionary(label_type=label_type)

2022-02-15 13:50:57,174 Computing label dictionary. Progress:


100%|██████████| 38219/38219 [00:03<00:00, 12704.04it/s]

2022-02-15 13:51:00,235 Corpus contains the labels: pos (#912344)
2022-02-15 13:51:00,237 Created (for label 'pos') Dictionary with 46 tags: <unk>, NNP, ,, CD, NNS, JJ, MD, VB, DT, NN, IN, ., VBZ, VBG, CC, VBD, VBN, RB, TO, PRP, RBR, WDT, VBP, RP, PRP$, JJS, POS, ``, EX, '', WP, :, JJR, WRB, $, NNPS, WP$, -LRB-, -RRB-, PDT, RBS, FW, UH, SYM, LS, #


# 3. Modelo Flair

In [10]:
stacked_embeddings = StackedEmbeddings([
                                        CharacterEmbeddings(),
                                        WordEmbeddings('en'),
                                        BytePairEmbeddings('en'),
                                        # FlairEmbeddings('news-forward'),
                                        # FlairEmbeddings('news-backward')
                                       ])

2022-02-15 13:51:06,796 https://flair.informatik.hu-berlin.de/resources/characters/common_characters not found in cache, downloading to /tmp/tmpffbhf1sg


100%|██████████| 2887/2887 [00:00<00:00, 959505.20B/s]

2022-02-15 13:51:07,146 copying /tmp/tmpffbhf1sg to cache at /root/.flair/datasets/common_characters
2022-02-15 13:51:07,148 removing temp file /tmp/tmpffbhf1sg


2022-02-15 13:51:18,380 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-news-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpvhr_i9pz


100%|██████████| 1200000128/1200000128 [01:21<00:00, 14803160.53B/s]

...

2022-02-15 13:52:47,853 copying /tmp/tmpyt9c20cq to cache at /root/.flair/embeddings/en-fasttext-news-300d-1M


2022-02-15 13:52:47,939 removing temp file /tmp/tmpyt9c20cq
downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs100000.model


100%|██████████| 1987533/1987533 [00:00<00:00, 3191064.45B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs100000.d50.w2v.bin.tar.gz


100%|██████████| 19357958/19357958 [00:01<00:00, 14238014.54B/s]


In [11]:
tagger = SequenceTagger(hidden_size=64,
                        embeddings=stacked_embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        # use_crf=True
                        )

In [19]:
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

In [ ]:
# 7. start training
trainer.train('resources/taggers/example-upos',
              embeddings_storage_mode='none',
              optimizer=Adamax,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=5
              )

2022-02-15 15:03:00,051 ----------------------------------------------------------------------------------------------------
2022-02-15 15:03:00,053 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_1): WordEmbeddings(
      'en'
      (embedding): Embedding(1000001, 300)
    )
    (list_embedding_2): BytePairEmbeddings(model=2-bpe-en-100000-50)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=450, out_features=450, bias=True)
  (rnn): LSTM(450, 64, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=48, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2022-02-15 15:03:00,055 ----------------------------------------------------------------------------------------------------
2022-02-15 15:03

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/resources/taggers/example-upos/best-model.pt /content/drive/MyDrive/UFRN/2021.2/PLN/best-model-2.pt

In [ ]:
from flair.models import SequenceTagger
model = SequenceTagger.load("/content/drive/MyDrive/UFRN/2021.2/PLN/best-model-2.pt")

2022-02-14 14:03:03,996 loading file /content/drive/MyDrive/UFRN/2021.2/PLN/best-model.pt


In [ ]:
result = model.evaluate(corpus.test, gold_label_type='pos', mini_batch_size=1, out_path=f"predictions.txt")

In [ ]:
print(result.detailed_results)


Results:
- F-score (micro) 0.9652
- F-score (macro) 0.8439
- Accuracy 0.9652

By class:
              precision    recall  f1-score   support

          NN     0.9649    0.9555    0.9602     17834
          IN     0.9690    0.9837    0.9763     13492
         NNP     0.9748    0.9614    0.9681     13177
          DT     0.9928    0.9930    0.9929     11183
          JJ     0.9164    0.9137    0.9150      8215
         NNS     0.9713    0.9815    0.9764      8061
           ,     1.0000    1.0000    1.0000      6876
           .     0.9998    0.9998    0.9998      5381
          CD     0.9840    0.9956    0.9898      4823
         VBD     0.9724    0.9408    0.9563      4561
          RB     0.9243    0.8956    0.9097      4405
          VB     0.9719    0.9390    0.9552      3573
          CC     0.9978    0.9932    0.9955      3250
          TO     0.9997    1.0000    0.9998      2913
         VBN     0.8588    0.9143    0.8857      2707
         VBZ     0.9770    0.9667    0.9718   